In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from pathlib import Path
import fastai
from fastai.vision import *

In [ ]:
PATH = Path('../input')
np.random.seed(42)

In [ ]:
batch_size = 64

In [ ]:
tfms = get_transforms(flip_vert=True, max_zoom=1.5 ,max_lighting=0.5)

In [ ]:
src = (ImageList.from_csv(csv_name='train.csv', path=PATH, folder='train/train')
            .split_by_rand_pct()
            .label_from_df(cols='has_cactus'))

In [ ]:
data = (src.transform(tfms,size=128)
          .databunch(bs=batch_size)
           .normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(9, 9))

In [ ]:
data.classes, data.c

In [ ]:
learn = cnn_learner(data, models.densenet161, metrics=accuracy, path='./')

In [ ]:
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
lr = 1e-3
learn.fit_one_cycle(5,slice(lr))

In [ ]:
learn.save('stage-1-dense161')

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(5, max_lr=slice(1e-5,lr/5))

In [ ]:
learn.save('stage-2-dense161')

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(7,6))

In [ ]:
interp.plot_confusion_matrix(figsize=(3, 3))

In [ ]:
data = (src.transform(tfms, size=128)
        .add_test_folder('test/test')
        .databunch()
        .normalize(imagenet_stats))

In [ ]:
learn.data = data

In [ ]:
probs, _ = learn.get_preds(ds_type=DatasetType.Test)

In [ ]:
ilst = data.test_ds.x

In [ ]:
fnames = [item.name for item in ilst.items]; fnames[:10]

In [ ]:
test_df = pd.DataFrame({'id': fnames, 'has_cactus': probs.argmax(1)}); test_df.head()

In [ ]:
test_df.to_csv('submission.csv', index=None)